il faut avoir le "présentation_champions_complets.csv" sinon l'echec sera cuisant 

In [ ]:
import requests
import polars as pl

def get_lol_champions():
    #1 Récupération du dernier numéro de patch
    try:
        version_url = "https://ddragon.leagueoflegends.com/api/versions.json"
        latest_version = requests.get(version_url).json()[0]
        print(f"Chargement des données pour le patch : {latest_version}")
    except Exception as e:
        print(f"Erreur lors de la récupération de la version : {e}")
        return

    #2. Télécharger les données avec l'URL dynamique
    url = f"https://ddragon.leagueoflegends.com/cdn/{latest_version}/data/en_US/champion.json"
    
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Erreur lors du téléchargement des champions (Status: {response.status_code})")
        return

    data = response.json()
    champions_dict = data['data']

    #3. On chope les infos (Ta logique existante)
    rows = []
    for champ_name, details in champions_dict.items():
        stats = details['stats']
        info = details['info']
        
        rows.append({
            "Name": champ_name,
            "Class": details['tags'][0],  # Rôle principal
            "AttackRange": stats['attackrange'],
            "MoveSpeed": stats['movespeed'],

            "HP": stats['hp'],
            "HPperLevel": stats['hpperlevel'],

            "hpregen": stats['hpregen'],
            "hpregenperlevel": stats['hpregenperlevel'],

            "Armor": stats['armor'],
            "ArmorperLevel": stats['armorperlevel'],
            
            "AttackDamage": stats['attackdamage'],
            "AttackDamageperLevel": stats['attackdamageperlevel'],

            "attackspeed": stats['attackspeed'],
            "attackspeedperlevel": stats['attackspeedperlevel'],

            "Spellblock": stats['spellblock'],
            "spellblockperlevel": stats['spellblockperlevel'],
            

            "Prv_attack": info['attack'],
            "Prv_defense": info['defense'],
            "Prv_magic": info['magic']
        })

    #4. Créer le Tableau + Polars
    tableau_champ = pl.DataFrame(rows)
    
    

    # Affichage 
    pl.Config.set_tbl_rows(10) 
    print(tableau_champ)
    #suuuuuper important pour la suite sinon c'est ko
    return tableau_champ

if __name__ == "__main__":
    tableau_champ = get_lol_champions()

Chargement des données pour le patch : 16.2.1
shape: (172, 19)
┌─────────┬──────────┬────────────┬───────────┬───┬────────────┬───────────┬───────────┬───────────┐
│ Name    ┆ Class    ┆ AttackRang ┆ MoveSpeed ┆ … ┆ spellblock ┆ Prv_attac ┆ Prv_defen ┆ Prv_magic │
│ ---     ┆ ---      ┆ e          ┆ ---       ┆   ┆ perlevel   ┆ k         ┆ se        ┆ ---       │
│ str     ┆ str      ┆ ---        ┆ i64       ┆   ┆ ---        ┆ ---       ┆ ---       ┆ i64       │
│         ┆          ┆ i64        ┆           ┆   ┆ f64        ┆ i64       ┆ i64       ┆           │
╞═════════╪══════════╪════════════╪═══════════╪═══╪════════════╪═══════════╪═══════════╪═══════════╡
│ Aatrox  ┆ Fighter  ┆ 175        ┆ 345       ┆ … ┆ 2.05       ┆ 8         ┆ 4         ┆ 3         │
│ Ahri    ┆ Mage     ┆ 550        ┆ 330       ┆ … ┆ 1.3        ┆ 3         ┆ 4         ┆ 8         │
│ Akali   ┆ Assassin ┆ 125        ┆ 345       ┆ … ┆ 2.05       ┆ 5         ┆ 3         ┆ 8         │
│ Akshan  ┆ Marksman ┆ 500  

In [12]:
import requests
import polars as pl
import pandas as pd # On a besoin de Pandas pour le ML et la compatibilité
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import joblib

def get_lol_champions_data():
    print("⏳ Récupération de la version du patch...")
    try:
        version_url = "https://ddragon.leagueoflegends.com/api/versions.json"
        latest_version = requests.get(version_url).json()[0]
        print(f"✅ Patch détecté : {latest_version}")
    except Exception as e:
        print(f"❌ Erreur version : {e}")
        return None

    url = f"https://ddragon.leagueoflegends.com/cdn/{latest_version}/data/en_US/champion.json"
    print(f"📥 Téléchargement des champions depuis : {url}")
    
    response = requests.get(url)
    if response.status_code != 200:
        print(f"❌ Erreur téléchargement (Status: {response.status_code})")
        return None

    data = response.json()
    champions_dict = data['data']

    rows = []
    for champ_name, details in champions_dict.items():
        stats = details['stats']
        info = details['info']
        
        # Construction de l'URL de l'image de chargement (pour l'affichage Streamlit)
        # Format : https://ddragon.leagueoflegends.com/cdn/img/champion/loading/Aatrox_0.jpg
        image_url = f"https://ddragon.leagueoflegends.com/cdn/img/champion/loading/{champ_name}_0.jpg"

        rows.append({
            "Name": champ_name,
            "Class": details['tags'][0],
            "Image_Loading": image_url,  # <--- AJOUT CRUCIAL POUR TON AFFICHAGE

            "AttackRange": stats['attackrange'],
            "MoveSpeed": stats['movespeed'],
            "HP": stats['hp'],
            "HPperLevel": stats['hpperlevel'],
            
            # Attention aux minuscules définies dans ton extraction
            "hpregen": stats['hpregen'],
            "hpregenperlevel": stats['hpregenperlevel'],
            
            "Armor": stats['armor'],
            "ArmorperLevel": stats['armorperlevel'],
            
            "AttackDamage": stats['attackdamage'],
            "AttackDamageperLevel": stats['attackdamageperlevel'],
            
            "attackspeed": stats['attackspeed'],
            "attackspeedperlevel": stats['attackspeedperlevel'],
            
            "Spellblock": stats['spellblock'],
            "spellblockperlevel": stats['spellblockperlevel'],

            "Prv_attack": info['attack'],
            "Prv_defense": info['defense'],
            "Prv_magic": info['magic']
        })

    # Conversion directe en Pandas pour Scikit-Learn
    return pd.DataFrame(rows)

# --- PROGRAMME PRINCIPAL ---

if __name__ == "__main__":
    
    # 1. Extraction
    df = get_lol_champions_data()
    
    if df is not None:
        print(f"📊 Données chargées : {len(df)} champions.")
        
        # 2. DEFINITION DES FEATURES (Doit matcher EXACTEMENT les clés ci-dessus)
        FEATURES = [
            "HP", "HPperLevel", 
            "MoveSpeed", 
            "Armor", "ArmorperLevel", 
            "Spellblock", "spellblockperlevel", 
            "AttackRange", 
            "hpregen", "hpregenperlevel", 
            "AttackDamage", "AttackDamageperLevel", 
            "attackspeed", "attackspeedperlevel", 
            "Prv_attack", "Prv_defense", "Prv_magic"
        ]

        # 3. Préparation
        X = df[FEATURES].fillna(0)
        
        # 4. Standardisation
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # 5. Entraînement KNN
        knn = NearestNeighbors(n_neighbors=4, metric='euclidean')
        knn.fit(X_scaled)

        # 6. Sauvegarde
        to_save = {
            'model': knn,
            'scaler': scaler,
            'data': df,       # Contient maintenant la colonne 'Image_Loading' !
            'features': FEATURES
        }

        joblib.dump(to_save, 'knn_data.pkl')
        print("✅ Fichier 'knn_data.pkl' généré avec succès !")
        print("👉 Tu peux maintenant relancer ton app Streamlit.")

⏳ Récupération de la version du patch...
✅ Patch détecté : 16.2.1
📥 Téléchargement des champions depuis : https://ddragon.leagueoflegends.com/cdn/16.2.1/data/en_US/champion.json
📊 Données chargées : 172 champions.
✅ Fichier 'knn_data.pkl' généré avec succès !
👉 Tu peux maintenant relancer ton app Streamlit.
